In [1]:
from tensorzero import AsyncTensorZero
import random

In [8]:
tensorzero_client = AsyncTensorZero("http://localhost:3000")

In [9]:
with open("nounlist.txt", "r") as file:
    nouns = [line.strip() for line in file]
    random.shuffle(nouns)


In [10]:
async def write_grade_haiku(topic: str, client: AsyncTensorZero) -> (str, bool):
    haiku_result = await client.inference(
        function_name="write_haiku",
        input={
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "value": {"topic": topic}}],
                }
            ]
        },
    )
    haiku_text = haiku_result.output[0].text
    print(haiku_text)
    haiku_lines = haiku_text.split("\n")
    haiku_text = "\n".join(haiku_lines[-3:])
    judge_result = await client.inference(
        function_name="judge_haiku",
        input={
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "value": {"topic": topic, "haiku": haiku_text}}
                    ],
                }
            ]
        },
    )
    score = judge_result.output.parsed["score"]
    # reasoning = judge_result.output.parsed["reasoning"]
    return haiku_text, score

In [11]:
for noun in nouns:
    haiku, score = await write_grade_haiku(noun, tensorzero_client)
    print(f"{noun}: \n{haiku}\n Passed: {score}")
    break

A reamer is a tool used in metalworking and woodworking to enlarge or finish a hole. Here are some key points about reamers:

1. Purpose: The primary purpose of a reamer is to create an accurate, smooth, and uniform hole size. It is used to enlarge, finish, or refine a pre-drilled hole.

2. Types of Reamers:
   - Straight reamers: Have a straight cutting surface and are used to enlarge or finish straight holes.
   - Taper reamers: Have a tapered cutting surface and are used to enlarge or finish tapered holes.
   - Expansion reamers: Have adjustable cutting edges that can be expanded to create holes of different sizes.

3. Materials: Reamers are typically made from high-speed steel (HSS) or carbide, which provide the necessary hardness and wear resistance for their cutting action.

4. Usage:
   - Reamers are used in a variety of applications, including metalworking, woodworking, and machining.
   - They are often used after drilling a hole to ensure a precise and consistent diameter.
  